In [1]:
"asa"

'asa'

In [1]:
# %load view1.py
import pandas as pd
import numpy as np
np.random.seed(1337)
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

C:\Users\Kshitij\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
img_rows, img_cols = 28,28

batch_size = 128 # Number of images used in each optimization step
nb_classes = 3 # One class per digit
nb_epoch = 1 # Number of times the whole data is used to learn

train = pd.read_csv("output_rotated.csv").values
#test  = pd.read_csv("test.csv").values

In [3]:
len(set(train[:,0]))

61578

In [4]:
# Check Keras backend
if(K.image_dim_ordering()=="th"):
    # Reshape the data to be used by a Theano CNN. Shape is
    # (nb_of_samples, nb_of_color_channels, img_width, img_heigh)
    X_train = train[:, 1:].reshape(train.shape[0], 1, img_rows, img_cols)
    X_test = test.reshape(test.shape[0], 1, img_rows, img_cols)
    in_shape = (1, img_rows, img_cols)
else:
    # Reshape the data to be used by a Tensorflow CNN. Shape is
    # (nb_of_samples, img_width, img_heigh, nb_of_color_channels)
    X_train = train[:, 2:].reshape(train.shape[0], img_rows, img_cols, 1)
#     X_test = test.reshape(test.shape[0], img_rows, img_cols, 1)
    in_shape = (img_rows, img_cols, 1)

In [5]:
y_train = train[:, 1] # First data is label (already removed from X_train)

for i in range(len(y_train)):
    y_train[i] -= 1

print(set(y_train))

# Make the value floats in [0;1] instead of int in [0;255]
X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')
X_train /= 255
# X_test /= 255

# convert class vectors to binary class matrices (ie one-hot vectors)
Y_train = np_utils.to_categorical(y_train, nb_classes)

#Display the shapes to check if everything's ok
print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)
# print('X_test shape:', X_test.shape)

{0, 1, 2}
X_train shape: (246312, 28, 28, 1)
Y_train shape: (246312, 3)


In [7]:
model = Sequential()
# For an explanation on conv layers see http://cs231n.github.io/convolutional-networks/#conv
# By default the stride/subsample is 1 and there is no zero-padding.
# If you want zero-padding add a ZeroPadding layer or, if stride is 1 use border_mode="same"
model.add(Convolution2D(12, 5, 5, activation = 'relu', input_shape=in_shape, init='he_normal'))

# For an explanation on pooling layers see http://cs231n.github.io/convolutional-networks/#pool
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(25, 5, 5, activation = 'relu', init='he_normal'))

model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the 3D output to 1D tensor for a fully connected layer to accept the input
model.add(Flatten())
model.add(Dense(180, activation = 'relu', init='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(100, activation = 'relu', init='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation = 'softmax', init='he_normal')) #Last layer with one output per class

# The function to optimize is the cross entropy between the true label and the output (softmax) of the model
# We will use adadelta to do the gradient descent see http://cs231n.github.io/neural-networks-3/#ada
model.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=["accuracy"])

# Make the model learn
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=10, verbose=1)

# Predict the label for X_test
# yPred = model.predict_classes(X_test)

C:\Users\Kshitij\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(12, (5, 5), activation="relu", input_shape=(28, 28, 1..., kernel_initializer="he_normal")`
  """
C:\Users\Kshitij\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(25, (5, 5), activation="relu", kernel_initializer="he_normal")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Kshitij\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(180, activation="relu", kernel_initializer="he_normal")`
  app.launch_new_instance()
C:\Users\Kshitij\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, activation="relu", kernel_initializer="he_normal")`
C:\Users\Kshitij\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your 

Epoch 1/10
246312/246312 [==============================] - 48s 194us/step - loss: 0.6547 - acc: 0.7260
Epoch 2/10
246312/246312 [==============================] - 48s 196us/step - loss: 0.5996 - acc: 0.7624
Epoch 3/10
246312/246312 [==============================] - 46s 187us/step - loss: 0.5856 - acc: 0.7690
Epoch 4/10
246312/246312 [==============================] - 43s 177us/step - loss: 0.5767 - acc: 0.7729
Epoch 5/10
246312/246312 [==============================] - 47s 189us/step - loss: 0.5710 - acc: 0.7758
Epoch 6/10
246312/246312 [==============================] - 46s 188us/step - loss: 0.5671 - acc: 0.7776
Epoch 7/10
246312/246312 [==============================] - 44s 179us/step - loss: 0.5624 - acc: 0.7795
Epoch 8/10
246312/246312 [==============================] - 43s 174us/step - loss: 0.5587 - acc: 0.7811
Epoch 9/10
246312/246312 [==============================] - 43s 175us/step - loss: 0.5553 - acc: 0.7824
Epoch 10/10
246312/246312 [==============================] - 180